## Import & constantes

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Affichage
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f')

# Dossier
RAW_DIR = "../data/raw"
PROC_DIR = "../data/processed"
os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(PROC_DIR, exist_ok=True)

# Paramètres dataset (mois de référence pour l'EDA)
DS_MONTH = "2023-01"
URL = f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{DS_MONTH}.parquet"
RAW_PATH = f"{RAW_DIR}/yellow_tripdata_{DS_MONTH}.parquet"

# Colonnes utiles (disponible sur 2023+)
USE_COLS = [
    "tpep_pickup_datetime", "tpep_dropoff_datetime",
    "passenger_count", "trip_distance",
    "PULocationID", "DOLocationID",
    "RatecodeID", "payment_type",
    "fare_amount", "extra", "mta_tax", 
    "tip_amount", "tolls_amount", "improvement_surcharge",
    "total_amount", "congestion_surcharge", "airport_fee"
]

## Téléchargement sécurisé (idempotent)

In [ ]:
import urllib.request

if not os.path.exists(RAW_PATH):
    print("Téléchargement du dataset...")
    urllib.request.urlretrieve(URL, RAW_PATH)
else:
    print("Déjà présent :", RAW_PATH)

## Chargment mémoire-friendly

In [ ]:
# Lecture partielle des colonnes pour réduire la RAM
df = pd.read_parquet(RAW_PATH, columns=USE_COLS)
df.shape, df.head(3)

## Typage & valeurs manquantes

In [ ]:
df.info()
df.isna().mean().sort_values(ascending=False).head(10)
df.describe(include='all').T.head(20)

## Cibles & features manquantes
On vise une régression **trip_duration_minutes**. 
**Hypothèse métier** (assumée et documentée):
`trip_distance` au moment de la prédiction provient d'un service de routage (proxy de la distance prévue). On ne l'a pas "vraiment" au pickup dans ce dataset, donc on **documente** l'approximation pour éviter la **data leakage confusion**. 

In [6]:
df["pickup_dt"] = pd.to_datetime(df["tpep_pickup_datetime"])
df["dropoff_dt"] = pd.to_datetime(df["tpep_dropoff_datetime"])
df["trip_duration_minutes"] = (df["dropoff_dt"] - df["pickup_dt"]).dt.total_seconds() / 60

# Features temporelles (utilisables au pickup)
df["pickup_hour"] = df["pickup_dt"].dt.hour
df["pickup_dow"] = df["pickup_dt"].dt.dayofweek  # 0=lundi
df["is_weekend"] = df["pickup_dow"].isin([5, 6]).astype(int)

## Règles d'hygiène (filtre simples)
Se débarasser des valeurs absurdes pour éviter d'entraîner un modèle sur du bruit. 

In [ ]:
# Bornes (conservatrices) pour éviter les outliers manifestes
mask = (
    (df["trip_duration_minutes"].between(1, 180, inclusive="both")) &
    (df["trip_distance"].between(0.1, 100, inclusive="both")) &
    (df["fare_amount"].between(0, 300, inclusive="both")) &
    (df["passenger_count"].fillna(1).between(1, 6, inclusive="both"))
)
clean = df[mask].copy()
clean.shape

**Pièges**
- Trop filtrer -> perdre la représentativité et biaiser la dérive. 
- Peu filtrer -> métriques gonflées artificiellement.
On ajuste plus finement après quelques visualisations. 

## Visuals EDA (rapides et utiles)

### Distributions clés

In [ ]:
plt.figure(figsize=(6,4)); sns.histplot(clean["trip_duration_minutes"], bins=60); plt.title("Durée (min)"); plt.show()
plt.figure(figsize=(6,4)); sns.histplot(clean["trip_distance"], bins=60); plt.title("Distance (miles)"); plt.show()
plt.figure(figsize=(6,4)); sns.histplot(clean["total_amount"], bins=60); plt.title("Total ($)"); plt.show()

### Relation distance <-> durée 

In [ ]:
sample = clean.sample(min(50000, len(clean)), random_state=42)
plt.figure(figsize=(6,6))
sns.scatterplot(data=sample, x="trip_distance", y="trip_duration_minutes", s=5, alpha=0.3)
plt.title("Distance vs Durée")
plt.show()

### Effet heure & jour

In [ ]:
fig, ax = plt.subplots(figsize=(7,4))
clean.groupby("pickup_hour")["trip_duration_minutes"].mean().plot(ax=ax)
ax.set_title("Durée moyenne par heure de pickup"); ax.set_xlabel("Heure"); ax.set_ylabel("Durée (min)")
plt.show()

fig, ax = plt.subplots(figsize=(7,4))
clean.groupby("pickup_dow")["trip_duration_minutes"].mean().plot(ax=ax)
ax.set_title("Durée moyenne par jour (0=lun)"); ax.set_xlabel("Jour"); ax.set_ylabel("Durée (min)")
plt.show()

**Bonnes pratiques**
- Garde ces graphiques "référence" pour la dérive temporelle (baselin = ce mois). 
- On figera un **profil de référence** (Evidently) à partir d'une **fenêtre d'entraînement**. 

## Échantillon "référence" sauvegardé
On sauve un **échantillon propre** qu'on versionnera (petit) -> reproductible.

In [ ]:
N = 200_000 if len(clean) >= 200_000 else len(clean)
ref = clean.sample(N, random_state=42)

FEATS_KEEP = [
    # target
    "trip_duration_minutes",
    #cfeatures utilisables en ligne (cf. hypothèse distance)
    "trip_distance", "passenger_count",
    "PULocationID", "DOLocationID",
    "pickup_hour", "pickup_dow", "is_weekend",
    "RatecodeID", "payment_type",
    # coûts (pour analyse/monitoring, pas nécessairement en features finales)
    "fare_amount", "tip_amount", "tolls_amount", "total_amount"
]
ref = ref[FEATS_KEEP].copy()

OUT_PATH = f"{PROC_DIR}/taxi_ref_{DS_MONTH}_clean_sample.parquet"
ref.to_parquet(OUT_PATH, index=False)
OUT_PATH, ref.shape, ref.head(3)

**À retenir**
- **data/raw/* non versionné, data/processed/*** oui (samples petits). 
- On documente clairement **l'hypothèse "distance prévue"** pour éviter tout procès en fuite. 